In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] =42
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
df1 = pd.read_csv("df_all.csv")
del df1['Unnamed: 0']
df1.head()

,Number,ratings2,ratings1,counts2,counts1,dev2,dev1,Text,length,word_leng
0,1,NaN,0.901997,0,32024.0,NaN,5.242998,"A man visits the doctor. The doctor says, I ha...",30,short
1,2,NaN,0.162989,0,35277.0,NaN,5.627972,This couple had an excellent relationship goin...,69,medium
2,3,NaN,0.193411,0,32402.0,NaN,5.448998,Q. Whats 200 feet long and has 4 teeth?A. The ...,17,short
3,4,NaN,-1.412599,0,30512.0,NaN,5.276202,Q. Whats the difference between a man and a to...,21,short
4,5,-2.034618,0.235352,566,73405.0,5.423289,5.302902,"Q. Whats O. J. Simpsons web address?A. Slash, ...",13,very short


In [5]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob

def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

vec = TfidfVectorizer(tokenizer=textblob_tokenizer,
                      stop_words='english',
                      max_df =0.9, # if you're in >90%, ignore
                      min_df =0.15, # if you're only in 15%, ignore
                      use_idf=True)

# Fit from the 'text' column of our dataframe
matrix = vec.fit_transform(df1['Text'])

# Then turn it into a new dataframe
results = pd.DataFrame(matrix.toarray(), columns=vec.get_feature_names())

CPU times: user 740 ms, sys: 5.55 ms, total: 745 ms
Wall time: 813 ms


In [6]:
results.head()

,ask,hi,look,man,repli,said,say,thi,wa
0,0.000000,0.000000,0.0,0.842112,0.401275,0.000000,0.360313,0.000000,0.000000
1,0.400093,0.350029,0.0,0.000000,0.409332,0.458754,0.000000,0.419115,0.404648
2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
%%time
from sklearn.cluster import KMeans

number_of_clusters=5
km = KMeans(n_clusters=number_of_clusters)

print("Fitting", number_of_clusters, "clusters usinga ", matrix.shape, "matrix")

km.fit(matrix)
km.fit

Fitting 5 clusters usinga  (150, 9) matrix
CPU times: user 147 ms, sys: 16.4 ms, total: 164 ms
Wall time: 234 ms


In [8]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vec.get_feature_names()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :5]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Top terms per cluster:
Cluster 0: look ask repli thi man
Cluster 1: say thi hi repli look
Cluster 2: hi look ask wa say
Cluster 3: man say ask repli wa
Cluster 4: wa said repli hi ask


In [9]:
df1.Text

0      A man visits the doctor. The doctor says, I ha...
1      This couple had an excellent relationship goin...
2      Q. Whats 200 feet long and has 4 teeth?A. The ...
3      Q. Whats the difference between a man and a to...
4      Q. Whats O. J. Simpsons web address?A. Slash, ...
5      Bill and Hillary Clinton are on a trip back to...
6      How many feminists does it take to screw in a ...
7      Q. Did you hear about the dyslexic devil worsh...
8      A country guy goes into a city bar that has a ...
9      Two cannibals are eating a clown. One turns to...
10     Q. What do a hurricane, a tornado, and a redne...
11     A guy stood over his tee shot for what seemed ...
12     They asked the Japanese visitor if they have e...
13     The father was very anxious to marry off his o...
14     Q:What did the blind person say when given som...
15     Q. What is orange and sounds like a parrot?A. ...
16     How many men does it take to screw in a light ...
17     A dog walks into Western

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob

def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

from sklearn.feature_extraction import stop_words
custom_stopwords = ['say', 'hi', 'ask', 'said'] + list(stop_words.ENGLISH_STOP_WORDS)

# Vectorize and save into a new dataframe
vec = TfidfVectorizer(stop_words=custom_stopwords,
                      max_df =0.9, # if you're in >90%, ignore
                      max_features = 1000,
                      use_idf=True)

# Fit from the 'text' column of our dataframe
matrix = vec.fit_transform(df1['Text'])

# Then turn it into a new dataframe
results = pd.DataFrame(matrix.toarray(), columns=vec.get_feature_names())

In [11]:
matrix.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [12]:
%%time
from sklearn.cluster import KMeans

# How many clusters?
number_of_clusters=10
km = KMeans(n_clusters=number_of_clusters)

print("Fitting", number_of_clusters, "clusters usinga ", matrix.shape, "matrix")

# Let's fit it!
km.fit(matrix)
km.fit

Fitting 10 clusters usinga  (150, 1000) matrix
CPU times: user 249 ms, sys: 2.66 ms, total: 252 ms
Wall time: 288 ms


In [13]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vec.get_feature_names()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :10]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Top terms per cluster:
Cluster 0: head line mickey tornado redneck time divorce 14 13 began
Cluster 1: room st peter lightbulb president phones languages just american phone
Cluster 2: says does br asks darwin long turns duck hillary eating
Cluster 3: difference whats slash sasquatch toilet engineers plumber government used sketch
Cluster 4: bulb light screw does men radio thats funny night presidents
Cluster 5: new work shredder replied come 00 asked station need japanese
Cluster 6: father school im little boy teacher goes man son mother
Cluster 7: engineer person did hell 10 just guy says parrot opposite
Cluster 8: woman man lost minutes beautiful pants ages atom watson time
Cluster 9: doctor man news says blah bad replies neutron cancer dont


In [14]:
km.labels_

array([9, 7, 3, 3, 3, 2, 4, 7, 7, 2, 0, 7, 5, 6, 7, 7, 4, 6, 1, 3, 3, 2, 5,
       1, 3, 8, 7, 7, 6, 3, 1, 1, 9, 6, 7, 7, 2, 7, 8, 1, 2, 8, 2, 2, 6, 8,
       7, 2, 7, 6, 5, 1, 6, 6, 6, 9, 1, 2, 7, 7, 1, 7, 1, 1, 0, 7, 6, 9, 6,
       5, 1, 1, 3, 4, 3, 8, 7, 3, 7, 2, 9, 2, 9, 3, 4, 9, 9, 9, 7, 2, 7, 6,
       7, 8, 7, 7, 1, 8, 5, 3, 1, 6, 9, 6, 0, 7, 5, 8, 5, 7, 5, 6, 5, 8, 4,
       8, 5, 9, 6, 5, 7, 7, 8, 7, 1, 8, 6, 1, 7, 9, 8, 0, 5, 9, 0, 6, 0, 8,
       7, 6, 7, 6, 7, 2, 0, 5, 0, 1, 6, 2], dtype=int32)

In [15]:
df1['group'] = pd.Series(km.labels_)

In [16]:
df1.head()

,Number,ratings2,ratings1,counts2,counts1,dev2,dev1,Text,length,word_leng,group
0,1,NaN,0.901997,0,32024.0,NaN,5.242998,"A man visits the doctor. The doctor says, I ha...",30,short,9
1,2,NaN,0.162989,0,35277.0,NaN,5.627972,This couple had an excellent relationship goin...,69,medium,7
2,3,NaN,0.193411,0,32402.0,NaN,5.448998,Q. Whats 200 feet long and has 4 teeth?A. The ...,17,short,3
3,4,NaN,-1.412599,0,30512.0,NaN,5.276202,Q. Whats the difference between a man and a to...,21,short,3
4,5,-2.034618,0.235352,566,73405.0,5.423289,5.302902,"Q. Whats O. J. Simpsons web address?A. Slash, ...",13,very short,3


In [17]:
df1.to_csv("df_groups.csv")

In [18]:
df1.group.value_counts().sum()

150

In [19]:
df1.group.value_counts()

7    33
6    21
1    17
8    14
2    14
9    13
5    13
3    12
0     8
4     5
Name: group, dtype: int64

In [23]:
df1.groupby('group').ratings1.mean().sort_values(ascending=False)

group
0    1.859555
8    1.374478
6    1.348745
9    1.268637
7    1.026428
1    0.540654
3    0.306800
2   -0.094567
5   -0.420371
4   -0.712265
Name: ratings1, dtype: float64

From best to worst rating:
Cluster 0: head line mickey tornado redneck time divorce 14 13 began
Cluster 8: woman man lost minutes beautiful pants ages atom watson time
Cluster 6: father school im little boy teacher goes man son mother
Cluster 9: doctor man news says blah bad replies neutron cancer dont
Cluster 7: engineer person did hell 10 just guy says parrot opposite
Cluster 1: room st peter lightbulb president phones languages just american phone
Cluster 3: difference whats slash sasquatch toilet engineers plumber government used sketch
Cluster 2: says does br asks darwin long turns duck hillary eating
Cluster 5: new work shredder replied come 00 asked station need japanese
Cluster 4: bulb light screw does men radio thats funny night presidents